In [1]:
library(knitr)
library(sqldf)
library(ggplot2)
library(mongolite)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(tidyr)
library(jsonlite)
library(lubridate)
library(gridExtra)
library(GGally)
library(scales)
library(memisc)
 library(sp)
library(rgeos)
library(rgdal)
library(maptools)
library(RColorBrewer)
library(maps)
library(grid)
library(cluster)
library(StatMatch)
library(fastcluster)
library(factoextra)
library(stringr)
library(googleVis)
library(RMySQL)
library(data.table)
library(dbplyr)
library(DBI)
library(httr)
library(scanstatistics)
library(purrr)
library(magrittr)
library(rsatscan)
library(stringi)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite

Attaching package: 'dplyr'

The following object is masked from 'package:gridExtra':

    combine

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'GGally'

The following object is masked from 'package:dplyr':

    nasa

Loading required package: lattice
Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select


Attaching package: 'memisc'

The following object is masked from 'package:scales':

    percent

The following object is masked from 'package:lubridate':

    is.interval

The following objects are masked from 'package:dplyr':

    collect, recode, rename, syms

The following objec

Washington DC is missing from tweet locations. This file adds the city and its demographic information

In [2]:
plot.format <- theme(axis.text = element_text(size = 14), legend.text = element_text(size = 14), legend.title = element_text(size = 14), axis.title = element_text(size = 18), plot.title = element_text(size = 18))

In [ ]:
#### Load location data for cities (latitude, longitude etc.)

In [63]:
cities_latlon <- read.csv("../Preprocessing/Files/Maps/cities_latlon.csv", stringsAsFactors = FALSE)

In [64]:
cities_latlon$X <- NULL
str(cities_latlon)

'data.frame':	28706 obs. of  15 variables:
 $ lon       : num  -124 -122 -118 -122 -123 ...
 $ lat       : num  47 48.7 48.4 47.3 46.6 ...
 $ city      : chr  "ABERDEEN" "ACME" "ADDY" "ADELAIDE" ...
 $ state     : chr  "WA" "WA" "WA" "WA" ...
 $ mapid     : int  1 2 3 4 5 6 7 8 9 10 ...
 $ area      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ perimeter : int  0 0 0 0 0 0 0 0 0 0 ...
 $ feature   : chr  "Population 10,000 to 49,999" "Population less than 10,000" "Population less than 10,000" "Population less than 10,000" ...
 $ name      : chr  "Aberdeen" "Acme" "Addy" "Adelaide" ...
 $ fips      : int  53027 53073 53065 53033 53041 53047 53077 53075 53053 53039 ...
 $ fips55    : int  100 275 380 415 590 660 800 1010 1045 1080 ...
 $ pop       : int  16565 0 0 0 0 0 0 632 0 0 ...
 $ state_fips: int  53 53 53 53 53 53 53 53 53 53 ...
 $ county    : chr  "Grays Harbor County" "Whatcom County" "Stevens County" "King County" ...
 $ location  : chr  "ABERDEEN WA" "ACME WA" "ADDY WA" "ADELAIDE WA" ...

In [ ]:
#### Location census data by US Place_type

In [65]:
usstates_place <- read.csv("../Preprocessing/Files/Demographics/usstates_placetype.csv", stringsAsFactors = FALSE)

In [66]:
usstates_place$X <- NULL
usstates_place$X.1 <- NULL
str(usstates_place)

'data.frame':	81542 obs. of  27 variables:
 $ rownum           : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Code             : chr  "AK" "AK" "AK" "AK" ...
 $ State            : chr  "Alaska" "Alaska" "Alaska" "Alaska" ...
 $ Sumlev           : int  40 162 162 162 162 162 162 162 162 162 ...
 $ State_id         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ County           : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Place            : int  0 65 650 870 1090 1200 1420 1860 1970 2080 ...
 $ Cousub           : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Concit           : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Primgeo_flag     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Funcstat         : chr  "A" "A" "A" "A" ...
 $ City             : chr  "Alaska" "Adak city" "Akhiok city" "Akiak city" ...
 $ Census2010pop    : int  710231 326 71 346 1027 677 219 105 258 324 ...
 $ Estimatesbase2010: int  710249 326 71 346 1027 677 220 171 258 324 ...
 $ Popestimate2010  : int  714015 326 71 347 1029 678 220 172 259 326 ...
 $ Popestimate2011  : int  722259 327 72

In [67]:
demo_cols_csv <- read.csv("../Preprocessing/Files/Demographics/acs_demo_cols2.csv", 
                         stringsAsFactors = FALSE)

In [68]:
demo_cols_csv$X <- NULL
str(demo_cols_csv)

'data.frame':	29321 obs. of  36 variables:
 $ rownum                      : int  1 2 3 4 5 6 7 8 9 10 ...
 $ id                          : chr  "1600000US0100100" "1600000US0100124" "1600000US0100460" "1600000US0100484" ...
 $ id2                         : int  100100 100124 100460 100484 100676 100820 100988 101132 101180 101228 ...
 $ geography                   : chr  "Abanda CDP, Alabama" "Abbeville city, Alabama" "Adamsville city, Alabama" "Addison town, Alabama" ...
 $ totalpopulation             : int  52 2646 4454 682 293 31905 21406 14787 3912 3727 ...
 $ malepop                     : int  11 1292 2269 331 143 15590 10777 6834 1774 1875 ...
 $ femalepop                   : int  41 1354 2185 351 150 16315 10629 7953 2138 1852 ...
 $ medianage                   : num  60.5 52.5 39.6 42.5 43.6 36.6 33.6 37.9 39.9 30.8 ...
 $ per_male                    : num  21.1 48.8 50.9 48.5 48.8 ...
 $ per_female                  : num  78.8 51.2 49.1 51.5 51.2 ...
 $ total_area             

In [ ]:
### Load demographic information

In [132]:
features <- read.csv("../Preprocessing/Files/Features/features2.csv")

In [5]:
nrow(features)

[1] 5223

In [70]:
features$X <- NULL
str(features)

'data.frame':	5223 obs. of  56 variables:
 $ Code                        : Factor w/ 49 levels "AK","AL","AR",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ State                       : Factor w/ 49 levels "Alabama","Alaska",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ State_id                    : int  2 2 2 2 2 2 2 2 2 2 ...
 $ City                        : Factor w/ 4393 levels "Abbeville city",..: 92 234 1298 1830 1996 2019 2061 2821 2857 3032 ...
 $ City2                       : Factor w/ 3994 levels "Abbeville","Aberdeen",..: 82 209 1177 1653 1804 1824 1865 2547 2578 2736 ...
 $ City3                       : Factor w/ 3994 levels "Abbeville","Aberdeen",..: 82 209 1177 1653 1804 1824 1865 2547 2578 2736 ...
 $ Place_type                  : Factor w/ 6 levels "borough","city",..: 4 2 2 2 2 2 2 2 2 2 ...
 $ City_state                  : Factor w/ 5218 levels "Abbeville, South Carolina",..: 111 279 1539 2188 2395 2419 2465 3403 3441 3639 ...
 $ City_code                   : Factor w/ 5218 levels "Abbeville, SC",

In [ ]:
#### Load English text already preprocessed

In [71]:
full_text_en <- fread(file="../Preprocessing/Files/Lexicon/full_text_en.csv", sep =",", header = TRUE, 
              stringsAsFactors=FALSE,
             showProgress=getOption("datatable.showProgress", interactive()), data.table=FALSE)

In [72]:
full_text_en$V1 <- NULL
str(full_text_en)

'data.frame':	3760412 obs. of  40 variables:
 $ rowno                      : num  1 2 3 4 5 6 7 8 9 10 ...
 $ user.id                    :integer64 1539100118 9439062 63380850 1539100118 9439062 26312359 40652267 296368494 ... 
 $ user.location              : chr  "Columbus, OH" "Atlanta, GA" "CaLove" "Columbus, OH" ...
 $ user.description           : chr  "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." "Husband. Father. Prelate. Human. Company builder, 2 exits. Author @kingonomics. Inclusive ecosystem builder @th"| __truncated__ "Jealous Guardian of our Democracy #RIPTrayvonMartin #ImAlwaysWithHer" "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." ...
 $ created_at                 : chr  "Sat Aug 09 22:33:06 +0000 2014" "Sat Aug 09 22:59:13 +0000 2014" "Sat Aug 09 22:00:02 +0000 2014" "Sat Aug 09 22:28:42 +0000 2014" ...
 $ id                         : num  4.98e+17 4.98e+17 4.98e+17 4.98e+17 4.98e+17 ...
 $ user.

In [46]:
### Check for Washington DC in the tweet locations

In [73]:
checkDC <- subset(full_text_en, user.location2 %like% "WASHINGTON DC")

In [74]:
head(checkDC$user.location2, 20)

[1] "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC"
 [5] "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC"
 [9] "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC"
[13] "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC"
[17] "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC" "WASHINGTON DC"

In [75]:
table(checkDC$City_code, useNA = "ifany")


 <NA> 
54941 

In [76]:
full_text_en$user.location2 <- gsub("WASHINGTON D C", "WASHINGTON DC",full_text_en$user.location2)

In [77]:
nrow(subset(full_text_en, user.location2 == "WASHINGTON DC"))

[1] 59813

In [78]:
full_text_en  <- full_text_en %>% 
                mutate(Code = ifelse(user.location2 == "WASHINGTON DC", 
                                 str_split_fixed(user.location2, " ", n = 2)[, 2],
                                 Code),
                     State = ifelse(user.location2 == "WASHINGTON DC", "DC", State),
                     City2 = ifelse(user.location2 == "WASHINGTON DC", 
                                    str_split_fixed(user.location2, " ", n = 2)[, 1], 
                                   City2),
                    City_code = ifelse(user.location2 == "WASHINGTON DC", 
                                       "WASHINGTON DC", City_code),
                    City_state = ifelse(user.location2 == "WASHINGTON DC", "WASHINGTON DC", 
                                        City_state)
                      )

In [79]:
nrow(full_text_en)

[1] 3760412

In [80]:
###### Add Washington DC to cities_latlon

In [81]:
checkDC <- subset(cities_latlon, name == "Washington D.C.")

In [82]:
checkDC$name <- gsub("\\.", "", toupper(checkDC$name))

In [83]:
checkDC$city <- str_split_fixed(checkDC$name, " ", n = 2)[, 1]
checkDC$state <- str_split_fixed(checkDC$name, " ", n = 2)[, 2]
checkDC$location <- str_c(checkDC$city, " ", checkDC$state)

In [84]:
colnames(checkDC)

[1] "lon"        "lat"        "city"       "state"      "mapid"     
 [6] "area"       "perimeter"  "feature"    "name"       "fips"      
[11] "fips55"     "pop"        "state_fips" "county"     "location"

In [85]:
cities_latlon[which(cities_latlon$name == "Washington D.C."), ] <- checkDC

In [86]:
checkDC

,lon,lat,city,state,mapid,area,perimeter,feature,name,fips,fips55,pop,state_fips,county,location
28703,-77.01257,38.88912,WASHINGTON,DC,28703,0,0,"Population 500,000 to 999,999 National Capital",WASHINGTON DC,11001,50000,606900,11,NA,WASHINGTON DC


In [87]:
subset(cities_latlon, name == "WASHINGTON DC")

,lon,lat,city,state,mapid,area,perimeter,feature,name,fips,fips55,pop,state_fips,county,location
28703,-77.01257,38.88912,WASHINGTON,DC,28703,0,0,"Population 500,000 to 999,999 National Capital",WASHINGTON DC,11001,50000,606900,11,NA,WASHINGTON DC


write.csv(cities_latlon, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Maps\\cities_latlondc.csv")

cities_latlon <- read.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Maps\\cities_latlondc.csv")

cities_latlon$X <- NULL
str(cities_latlon)

In [89]:
full_text_en <-  full_text_en %>% 
                left_join(cities_latlon[c("lon", "lat", "city", "state")], 
                          by = c("City2" = "city", "Code" = "state")) %>%
                as.data.frame()

In [90]:
get_num_tweets <- full_text_en %>% dplyr::select(one_of(c("ymd", "yday", "City_code"))) %>%
                                    group_by(ymd, yday, City_code) %>%
                                            summarise(n2 = n()) %>%
                as.data.frame()

In [38]:
str(get_num_tweets)

'data.frame':	28438 obs. of  4 variables:
 $ ymd      : chr  "2014-08-09" "2014-08-09" "2014-08-09" "2014-08-09" ...
 $ yday     : int  221 221 221 221 221 221 221 221 221 221 ...
 $ City_code: chr  "ATLANTA GA" "AUSTIN TX" "BALTIMORE MD" "BEAVERTON OR" ...
 $ n2       : int  7 2 2 2 2 1 2 1 4 1 ...


In [91]:
colnames(full_text_en)

[1] "rowno"                       "user.id"                    
 [3] "user.location"               "user.description"           
 [5] "created_at"                  "id"                         
 [7] "user.created_at"             "retweeted"                  
 [9] "retweeted_count"             "retweet"                    
[11] "retweeted_status.created_at" "favorited"                  
[13] "favorite_count"              "textnum"                    
[15] "full_text"                   "possibly_sensitive"         
[17] "hashtags"                    "lang"                       
[19] "user.geo_enabled"            "place.full_name"            
[21] "place.place_type"            "utc_datetime"               
[23] "yday"                        "ymd"                        
[25] "user_id2"                    "user.location2"             
[27] "City_code"                   "City_state"                 
[29] "City2"                       "State"                      
[31] "Code"                        "n"                          
[33] "keyword_pos_hash"            "keyword_count_hash"         
[35] "full_text2"                  "full_text_nohash"           
[37] "tweet_type"                  "full_text_nohash2"          
[39] "full_text_low"               "full_text_low_nostop"       
[41] "lon"                         "lat"

In [92]:
full_text_en <- full_text_en %>% 
                inner_join(get_num_tweets, by =c("ymd", "yday", "City_code")) %>%
                dplyr::select(-n) %>%
                dplyr::rename("n" = "n2") %>%
                as.data.frame()

In [93]:
subset(full_text_en[c("City_code", "lon", "lat")], City_code == "WASHINGTON DC")

,City_code,lon,lat
106,WASHINGTON DC,-77.01257,38.88912
192,WASHINGTON DC,-77.01257,38.88912
213,WASHINGTON DC,-77.01257,38.88912
220,WASHINGTON DC,-77.01257,38.88912
238,WASHINGTON DC,-77.01257,38.88912
247,WASHINGTON DC,-77.01257,38.88912
270,WASHINGTON DC,-77.01257,38.88912
284,WASHINGTON DC,-77.01257,38.88912
406,WASHINGTON DC,-77.01257,38.88912
430,WASHINGTON DC,-77.01257,38.88912


In [ ]:
### Placeholder to read and write new full_text_en file with Washington DC city added as 
### a location

write.csv(full_text_en, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Lexicon\\full_text_en2_dc.csv")

full_text_en2 <- fread(file="C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Lexicon\\full_text_en2_dc.csv", sep =",", header = TRUE, 
              stringsAsFactors=FALSE,
             showProgress=getOption("datatable.showProgress", interactive()), data.table=FALSE)

In [10]:
str(full_text_en2)
full_text_en2$V1 <- NULL

'data.frame':	3760412 obs. of  43 variables:
 $ V1                         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ rowno                      : num  1 2 3 4 5 6 7 8 9 10 ...
 $ user.id                    :integer64 1539100118 9439062 63380850 1539100118 9439062 26312359 40652267 296368494 ... 
 $ user.location              : chr  "Columbus, OH" "Atlanta, GA" "CaLove" "Columbus, OH" ...
 $ user.description           : chr  "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." "Husband. Father. Prelate. Human. Company builder, 2 exits. Author @kingonomics. Inclusive ecosystem builder @th"| __truncated__ "Jealous Guardian of our Democracy #RIPTrayvonMartin #ImAlwaysWithHer" "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." ...
 $ created_at                 : chr  "Sat Aug 09 22:33:06 +0000 2014" "Sat Aug 09 22:59:13 +0000 2014" "Sat Aug 09 22:00:02 +0000 2014" "Sat Aug 09 22:28:42 +0000 2014" ...
 $ id                         : 

In [136]:
NROW(unique(full_text_en$full_text_low_nostop))

[1] 850179

In [94]:
washingtonDC <- subset(demo_cols_csv, geography == "Washington city, District of Columbia")

In [ ]:
### Add missing Washington DC info by reordering columns in features and demo_cols_csv to 
### match

In [98]:
features <- features %>% dplyr::select(Code:house_median_income, 
                                med_per_urban:state, location:count_rate) %>%
                        as.data.frame()
                        

In [99]:
colnames(demo_cols_csv)

[1] "rownum"                       "id"                          
 [3] "id2"                          "geography"                   
 [5] "totalpopulation"              "malepop"                     
 [7] "femalepop"                    "medianage"                   
 [9] "per_male"                     "per_female"                  
[11] "total_area"                   "urban"                       
[13] "rural"                        "per_urban"                   
[15] "per_rural"                    "house_total_population"      
[17] "mean_income_per_capita"       "house_total_income"          
[19] "total_population_race"        "white_population"            
[21] "black_population"             "per_white"                   
[23] "per_black"                    "total_pop_over16"            
[25] "employ_pop_over16"            "per_employ_over16"           
[27] "total_population_lang"        "english_only"                
[29] "english_other"                "per_english_only"            
[31] "per_english_other"            "state"                       
[33] "city"                         "mean_income_households"      
[35] "total_population_individuals" "house_median_income"

In [112]:
usplaceDC <- c("DC", "DC", 11, "Washington city", "Washington", "Washington", "city", 
            "WASHINGTON DC", "WASHINGTON DC")
usplaceDC <- c(usplaceDC, washingtonDC$totalpopulation, washingtonDC$totalpopulation)
#colnames(usdemoDC) <- colnames(demo_cols_csv)[1:11]

In [113]:
usplaceDC <- as.data.frame(t(usplaceDC))

In [114]:
colnames(usplaceDC) <- colnames(features)[1:11]

In [103]:
str(usplaceDC)

'data.frame':	1 obs. of  11 variables:
 $ Code              : Factor w/ 1 level "DC": 1
 $ State             : Factor w/ 1 level "DC": 1
 $ State_id          : Factor w/ 1 level "11": 1
 $ City              : Factor w/ 1 level "Washington city": 1
 $ City2             : Factor w/ 1 level "Washington": 1
 $ City3             : Factor w/ 1 level "Washington": 1
 $ Place_type        : Factor w/ 1 level "city": 1
 $ City_state        : Factor w/ 1 level "WASHINGTON DC": 1
 $ City_code         : Factor w/ 1 level "WASHINGTON DC": 1
 $ avgCensus2010pop  : Factor w/ 1 level "647484": 1
 $ avgPopestimate2015: Factor w/ 1 level "647484": 1


In [115]:
usplaceDC[c("State_id","avgCensus2010pop","avgPopestimate2015")] <- apply(usplaceDC[c("State_id","avgCensus2010pop","avgPopestimate2015")],
                   2, as.character)
usplaceDC[c("State_id","avgCensus2010pop","avgPopestimate2015")] <- apply(usplaceDC[c("State_id","avgCensus2010pop","avgPopestimate2015")],
                   2, as.integer)

In [116]:
demo_cols_csv <- demo_cols_csv %>% dplyr::select(rownum:per_english_other, 
                                                 mean_income_households:house_median_income,
                                                 city, state) %>%
                                    as.data.frame()
                                    

In [106]:
colnames(demo_cols_csv)

[1] "rownum"                       "id"                          
 [3] "id2"                          "geography"                   
 [5] "totalpopulation"              "malepop"                     
 [7] "femalepop"                    "medianage"                   
 [9] "per_male"                     "per_female"                  
[11] "total_area"                   "urban"                       
[13] "rural"                        "per_urban"                   
[15] "per_rural"                    "house_total_population"      
[17] "mean_income_per_capita"       "house_total_income"          
[19] "total_population_race"        "white_population"            
[21] "black_population"             "per_white"                   
[23] "per_black"                    "total_pop_over16"            
[25] "employ_pop_over16"            "per_employ_over16"           
[27] "total_population_lang"        "english_only"                
[29] "english_other"                "per_english_only"            
[31] "per_english_other"            "mean_income_households"      
[33] "total_population_individuals" "house_median_income"         
[35] "city"                         "state"

In [107]:
colnames(features)

[1] "Code"                         "State"                       
 [3] "State_id"                     "City"                        
 [5] "City2"                        "City3"                       
 [7] "Place_type"                   "City_state"                  
 [9] "City_code"                    "avgCensus2010pop"            
[11] "avgPopestimate2015"           "rownum"                      
[13] "id"                           "id2"                         
[15] "geography"                    "totalpopulation"             
[17] "malepop"                      "femalepop"                   
[19] "medianage"                    "per_male"                    
[21] "per_female"                   "total_area"                  
[23] "urban"                        "rural"                       
[25] "per_urban"                    "per_rural"                   
[27] "house_total_population"       "mean_income_per_capita"      
[29] "house_total_income"           "total_population_race"       
[31] "white_population"             "black_population"            
[33] "per_white"                    "per_black"                   
[35] "total_pop_over16"             "employ_pop_over16"           
[37] "per_employ_over16"            "total_population_lang"       
[39] "english_only"                 "english_other"               
[41] "per_english_only"             "per_english_other"           
[43] "mean_income_households"       "total_population_individuals"
[45] "house_median_income"          "med_per_urban"               
[47] "med_per_rural"                "med_per_employ_over16"       
[49] "city"                         "state"                       
[51] "location"                     "lon"                         
[53] "lat"                          "count"                       
[55] "count_rate"

In [117]:
demo_cols_csvDC <- demo_cols_csv %>% dplyr::select(everything()) %>%
                    filter(city == "Washington city" & state == "District of Columbia") %>%
                    mutate(med_per_urban = per_urban, 
                           med_per_rural = per_rural, 
                          med_per_employ_over16 = per_employ_over16,
                          state = "DC",
                          city = "WASHINGTON") %>%
                    as.data.frame()

In [118]:
DCInfo <- dplyr::bind_cols(usplaceDC, demo_cols_csvDC)

In [119]:
str(DCInfo)

'data.frame':	1 obs. of  50 variables:
 $ Code                        : Factor w/ 1 level "DC": 1
 $ State                       : Factor w/ 1 level "DC": 1
 $ State_id                    : int 11
 $ City                        : Factor w/ 1 level "Washington city": 1
 $ City2                       : Factor w/ 1 level "Washington": 1
 $ City3                       : Factor w/ 1 level "Washington": 1
 $ Place_type                  : Factor w/ 1 level "city": 1
 $ City_state                  : Factor w/ 1 level "WASHINGTON DC": 1
 $ City_code                   : Factor w/ 1 level "WASHINGTON DC": 1
 $ avgCensus2010pop            : int 647484
 $ avgPopestimate2015          : int 647484
 $ rownum                      : int 4133
 $ id                          : chr "1600000US1150000"
 $ id2                         : int 1150000
 $ geography                   : chr "Washington city, District of Columbia"
 $ totalpopulation             : int 647484
 $ malepop                     : int 306674


cities_latlondc <- read.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Maps\\cities_latlondc.csv", stringsAsFactors = FALSE)

str(cities_latlondc)
cities_latlondc$X <- NULL

In [120]:
get_num_tweetsDC <- full_text_en %>% dplyr::select(one_of(c("City_code"))) %>%
                                    filter(City_code == "WASHINGTON DC") %>%
                                    group_by(City_code) %>%
                                            summarise(count = n()) %>%
                                    as.data.frame()

In [121]:
round(get_num_tweetsDC$count/DCInfo$totalpopulation * 1000, 2)

[1] 92.38

In [122]:
round(5/8, 3)

[1] 0.625

In [124]:
DCInfo <- DCInfo %>% dplyr::select(everything()) %>%
                        filter(city == "WASHINGTON", state == "DC") %>%
                        inner_join(cities_latlon[c("lon", "lat", "location", "city", "state")], 
                                  by = c("city", "state")) %>%
                        inner_join(get_num_tweetsDC[c("City_code", "count")], by = "City_code") %>%
                        as.data.frame()

Warning message:
"Column `City_code` joining factor and character vector, coercing into character vector"

In [125]:
DCInfo$count_rate <- round((DCInfo$count/DCInfo$totalpopulation) * 1000, 2)

In [126]:
DCInfo <- DCInfo %>% dplyr::select(Code:house_median_income, 
                                   med_per_urban:med_per_employ_over16, 
                                  city:state, location, lon:lat, count:count_rate) %>%
                    as.data.frame()

In [127]:
str(DCInfo)

'data.frame':	1 obs. of  55 variables:
 $ Code                        : Factor w/ 1 level "DC": 1
 $ State                       : Factor w/ 1 level "DC": 1
 $ State_id                    : int 11
 $ City                        : Factor w/ 1 level "Washington city": 1
 $ City2                       : Factor w/ 1 level "Washington": 1
 $ City3                       : Factor w/ 1 level "Washington": 1
 $ Place_type                  : Factor w/ 1 level "city": 1
 $ City_state                  : Factor w/ 1 level "WASHINGTON DC": 1
 $ City_code                   : chr "WASHINGTON DC"
 $ avgCensus2010pop            : int 647484
 $ avgPopestimate2015          : int 647484
 $ rownum                      : int 4133
 $ id                          : chr "1600000US1150000"
 $ id2                         : int 1150000
 $ geography                   : chr "Washington city, District of Columbia"
 $ totalpopulation             : int 647484
 $ malepop                     : int 306674
 $ femalepop     

In [128]:
DCInfo[c("Code", "State", "City", "City2", "City3", "City_state", "Place_type",
         "id", "geography")] <- 
apply(DCInfo[c("Code", "State", "City", "City2", "City3", "City_state", "Place_type",
                "id", "geography")], 2, as.character)

In [129]:
str(DCInfo)

'data.frame':	1 obs. of  55 variables:
 $ Code                        : chr "DC"
 $ State                       : chr "DC"
 $ State_id                    : int 11
 $ City                        : chr "Washington city"
 $ City2                       : chr "Washington"
 $ City3                       : chr "Washington"
 $ Place_type                  : chr "city"
 $ City_state                  : chr "WASHINGTON DC"
 $ City_code                   : chr "WASHINGTON DC"
 $ avgCensus2010pop            : int 647484
 $ avgPopestimate2015          : int 647484
 $ rownum                      : int 4133
 $ id                          : chr "1600000US1150000"
 $ id2                         : int 1150000
 $ geography                   : chr "Washington city, District of Columbia"
 $ totalpopulation             : int 647484
 $ malepop                     : int 306674
 $ femalepop                   : int 340810
 $ medianage                   : num 33.7
 $ per_male                    : num 47.4
 $ per_f

In [133]:
features <- features %>% bind_rows(DCInfo)

Warning message in bind_rows_(x, .id):
"binding factor and character vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding character and factor vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding factor and character vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding character and factor vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding factor and character vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding character and factor vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding factor and character vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding character and factor vector, coercing into character vector"Warning message in bind_rows_(x, .id):
"binding factor and character vector, coercing into character vector"Warning message in bind_rows

features <- write.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Features_Demo\\features2dc.csv", 
                     stringsAsFactors = FALSE)